# 程式環境參數設定與匯入套件

In [1]:
import sys

from numpy.core.fromnumeric import mean
sys.path.append("/Users/huangguanlun/pyProjects/quotoFuture/")
sys.path.append("/Users/huangguanlun/pyProjects/quotoFuture/qfVenv/lib/python3.8/site-packages/")

In [2]:
import simContainer
import numpy as np
from scipy.linalg import cholesky, inv

In [3]:
FLOAT_FORMATTER="{:.5f}".format
np.set_printoptions(formatter={'float_kind':FLOAT_FORMATTER})

# 宣告與定義模擬相關的函數、物件
## 物件 z0Container 的功用為抽取無相關的標準常態分配樣本，作為之後模擬路徑所用

In [4]:
class z0Container():
    def __init__(self, shape, varCntAxis):
        self.shape = shape
        self.varCntAxis = varCntAxis
        self.container = np.random.normal(
            loc = 0.0,
            scale = 1.0,
            size = self.shape
        )
    
    def meanMatch(self, get=False):
        tempMean = self.container.mean(axis = 2).reshape(self.shape[0], self.shape[1], 1, self.shape[3])
        self.container -= tempMean
        if get:
            return self.container

    def invCholesky(self, get=False):
        self.meanMatch()
        for i, trial in enumerate(self.container):
            for j, simPath in enumerate(self.container[i]):
                cov = np.cov(simPath, rowvar=False)
                covUtriInv = inv(cholesky(cov, lower=False))
                self.container[i][j] = simPath.dot(covUtriInv)
        if get:
            return self.container

    def reset(self):
        self.container = np.random.normal(
            loc = 0.0,
            scale = 1.0,
            size = self.shape
        )
        self.invCholesky(get= False)

## 物件 pathSim 為主要負責模擬路徑的模組，使用者輸入商品相關的參數、模擬路徑的數量以及重複試驗的次數後，再呼叫其方法便可進行股價路徑模擬。

In [5]:
class pathSim():
    def __init__(self, S0Arr, muArr, riskFreeRate, timePeriod, timePartitionCnt, qArr, sigmaArr, corrMatrx, simCnt, repeatCnt):
        self.lnS0Arr = np.log(np.array(S0Arr))
        self.muArr = np.array(muArr)
        self.assetCnt = self.lnS0Arr.shape[0]
        self.riskFreeRate = riskFreeRate
        self.timePeriod = timePeriod
        self.timePartitionCnt = timePartitionCnt
        self.qArr = np.array(qArr)
        self.sigmaArr = np.array(sigmaArr)
        self.corrMtrx = np.array(corrMatrx)
        self.simCnt = simCnt
        self.repeatCnt = repeatCnt

        # compute the distribution parameter of return in each delta_t period
        self.delta_t = self.timePeriod / self.timePartitionCnt
        self.mu_delta_t_return = (self.riskFreeRate - self.qArr - np.power(self.sigmaArr, 2) / 2) * (self.delta_t)
        self.sigma_delta_t_return = (self.sigmaArr * np.power(self.delta_t, 0.5))
        self.covMtrx = self.corrMtrx * np.tensordot(self.sigma_delta_t_return.reshape(1, -1), self.sigma_delta_t_return.reshape(1, -1), axes=(0,0)) 

        # draw from Z
        self.Z0Container = z0Container([self.repeatCnt, self.simCnt, self.timePartitionCnt, self.assetCnt], varCntAxis= 3)
        self.Z0 = self.Z0Container.meanMatch(get= True)

        self.returnSeries = np.zeros_like(self.Z0)

    def var_cov(self):
        utri = cholesky(self.covMtrx)
        # var-cov
        self.returnSeries = np.tensordot(self.Z0, utri, axes = ([3], [0]))

    def muShift(self):
        # mu shift
        self.returnSeries += self.mu_delta_t_return

    def lnS0Append(self):
        # append lnS0
        self.lnS0ArrPanel = self.lnS0Arr * np.ones((self.repeatCnt, self.simCnt, 1, self.assetCnt))
        self.lnStSeries = np.concatenate((self.lnS0ArrPanel, self.returnSeries), axis= 2)
    
    def cumsum(self):
        # cumsum
        self.lnStSeries = self.lnStSeries.cumsum(axis= 2)
        self.stSeries = np.exp(self.lnStSeries)

# 商品參數設定與變數宣告

### 股價一開始為 100 (USD), 匯率為 1/30 USD/NTD

In [6]:
S0Arr = [100, 1/30]

### 隨機過程參數 mu 分別為 0.25, 0.02, sigma 分別為 0.03, 0.05, 兩者報酬率之相關係數為 0.4

In [7]:
muArr = [0.25, 0.02]
sigmaArr = [0.03, 0.05]
corrMatrx = [[1, 0.4], [0.4, 1]]

### 美國與台灣之無風險利率皆為 3% 且為常數

In [8]:
riskFreeRate = 0.03

### 不發放股利

In [9]:
qArr = [0, 0]

### 到期時間為半年後, 模擬時切成 200 段, 一次試驗模擬 10000 條路徑, 重複做 10 次試驗, 共模擬 100000 條路徑

In [10]:
timePeriod = 0.5
timePartitionCnt = 200
simCnt = 10000
repeatCnt = 10

In [11]:
test = pathSim(
    S0Arr= S0Arr,
    muArr = muArr,
    riskFreeRate= riskFreeRate,
    timePartitionCnt= timePartitionCnt,
    timePeriod= timePeriod,
    qArr= qArr,
    sigmaArr= sigmaArr,
    corrMatrx= corrMatrx,
    simCnt= simCnt,
    repeatCnt= repeatCnt
)

# 執行股價路徑模擬

In [12]:
test.var_cov()
test.muShift()
test.lnS0Append()
test.cumsum()

# 依照股價路徑計算 carry cost
## 我們將 carry cost 分成兩部分，第一部分是建立部位後，不斷 roll over 所需要的成本。

In [13]:
posValSeries = test.stSeries[..., 0] / test.stSeries[..., 1]
plSeries = np.diff(posValSeries, axis = 2)
deltaTCnt = np.ones_like(plSeries) * np.arange(plSeries.shape[2])[::-1]
extraCarryCostSeries= plSeries * np.exp(riskFreeRate * deltaTCnt * (timePeriod / timePartitionCnt))
extraCarryCost = extraCarryCostSeries.sum(axis = 2)
meanExtraCarryCost = extraCarryCost.mean(axis = 1).mean(axis = 0)

In [14]:
meanExtraCarryCost

1.2144537178570514

## 第二部分為一開始建立部位後的成本。

In [15]:
initCost = (S0Arr[0] / S0Arr[1]) * np.exp(riskFreeRate * timePeriod)

## carry cost 的總和即為模擬出來的商品價格

In [16]:
simAns = meanExtraCarryCost + initCost
print(simAns)

3046.5536475650138


# 以公式驗證模擬結果
## 首先計算公式需要用到的，在Ｔ時刻，股價與匯率的共變異數

In [17]:
var_T = test.sigmaArr * np.power(test.timePeriod, 0.5).reshape(1, -1)
var_T

array([[0.02121, 0.03536]])

In [18]:
covMtrx = test.corrMtrx * np.tensordot(var_T, var_T, axes= (0, 0))
covMtrx

array([[0.00045, 0.00030],
       [0.00030, 0.00125]])

## 公式的計算結果為本國貨幣的價格，須經由t=0時刻之匯率轉換成外幣的價格

In [19]:
formulaSol = S0Arr[0] * np.exp(riskFreeRate * timePeriod) * np.exp(covMtrx[0][1])
print(formulaSol)

101.54176442197598


In [20]:
formulaSol / S0Arr[1]

3046.2529326592794

# 比較結果 

In [21]:
print(f"模擬價格：{simAns}, 公式解：{formulaSol/S0Arr[1]}")

模擬價格：3046.5536475650138, 公式解：3046.2529326592794
